In [ ]:
import get_price.presave_price as pp
pp.generate_and_update_futures_data_files(ticker_head_list='butterfly')

# probably better idea here is to check for spikes es you enter the trade
# and don't exit unless stop is triggered for 5 days consecutively


In [1]:
# Creating Date List For Backtesting
import sys
sys.path.append(r'C:\Users\kocat_000\quantFinance\PycharmProjects')
import backtesting.utilities as bu
dates_output = bu.get_backtesting_dates(date_to=20161014,years_back=10)
double_dates = sorted(dates_output['double_dates'],reverse=False)
selected_dates = double_dates
selected_dates


[20061018,
 20061025,
 20061101,
 20061108,
 20061115,
 20061122,
 20061129,
 20061206,
 20061213,
 20061220,
 20061227,
 20070103,
 20070110,
 20070117,
 20070124,
 20070131,
 20070207,
 20070214,
 20070221,
 20070228,
 20070307,
 20070314,
 20070321,
 20070328,
 20070404,
 20070411,
 20070418,
 20070425,
 20070502,
 20070509,
 20070516,
 20070523,
 20070530,
 20070606,
 20070613,
 20070620,
 20070627,
 20070711,
 20070718,
 20070725,
 20070801,
 20070808,
 20070815,
 20070822,
 20070829,
 20070905,
 20070912,
 20070919,
 20070926,
 20071003,
 20071010,
 20071017,
 20071024,
 20071031,
 20071107,
 20071114,
 20071121,
 20071128,
 20071205,
 20071212,
 20071219,
 20071226,
 20080102,
 20080109,
 20080116,
 20080123,
 20080130,
 20080206,
 20080213,
 20080220,
 20080227,
 20080305,
 20080312,
 20080319,
 20080326,
 20080402,
 20080409,
 20080416,
 20080423,
 20080430,
 20080507,
 20080514,
 20080521,
 20080528,
 20080604,
 20080611,
 20080618,
 20080625,
 20080702,
 20080709,
 20080716,

In [3]:
# Historical Butterfly Report Run:

import opportunity_constructs.futures_butterfly as fb

for date_to in selected_dates:
    fb.generate_futures_butterfly_sheet_4date(date_to=date_to)
    

    

In [ ]:
import backtesting.backtest_futures_with_paths as bfp
butterfly_output = bfp.get_backtest_summary(date_list = selected_dates,
                                            use_existing_filesQ=True)

big_data = butterfly_output['big_data']
backtest_output = butterfly_output['backtest_output']
big_data['mom5'] = big_data['recent_5day_pnl']/(abs(big_data['downside'])+big_data['upside'])



In [2]:
# Calculate Backtesting Results With A Variety of Stoploss Choices:

import backtesting.backtest_futures_butterfly as bfb

backtest_results_list = []

for i in range(len(selected_dates)):
    print(selected_dates[i])
    backtest_results_list.append(bfb.save_trade_paths(report_date=selected_dates[i],use_existing_filesQ=False))

    

20061018
20061025
20061101
20061108
20061115
20061122
20061129
20061206
20061213
20061220
20061227
20070103
20070110
20070117
20070124
20070131
20070207
20070214
20070221
20070228
20070307
20070314
20070321
20070328
20070404
20070411
20070418
20070425
20070502
20070509
20070516
20070523
20070530
20070606
20070613
20070620
20070627
20070711
20070718
20070725
20070801
20070808
20070815
20070822
20070829
20070905
20070912
20070919
20070926
20071003
20071010
20071017
20071024
20071031
20071107
20071114
20071121
20071128
20071205
20071212
20071219
20071226
20080102
20080109
20080116
20080123
20080130
20080206
20080213
20080220
20080227
20080305
20080312
20080319
20080326
20080402
20080409
20080416
20080423
20080430
20080507
20080514
20080521
20080528
20080604
20080611
20080618
20080625
20080702
20080709
20080716
20080723
20080730
20080806
20080813
20080820
20080827
20080903
20080910
20080917
20080924
20081001
20081008
20081015
20081022
20081029
20081105
20081112
20081119
20081126
20081203
2

In [4]:
import pandas as pd
big_data = pd.concat(backtest_results_list)
big_data

,ticker1,ticker2,ticker3,tickerHead,tickerClass,trDte1,trDte2,trDte3,multiplier,agg,...,short_stop_days1,short_stop_days2,short_stop_days3,long_pnl20,long_pnl40,long_pnl60,short_pnl20,short_pnl40,short_pnl60,report_date
0,LNZ2006,LNG2007,LNJ2007,LN,Livestock,40,81,122,400,12,...,NaN,NaN,NaN,-3114.000,NaN,NaN,3114.000,NaN,NaN,20061018
1,LNG2007,LNJ2007,LNM2007,LN,Livestock,81,122,164,400,12,...,NaN,NaN,NaN,-3468.000,-4896.000,5888.80,3876.000,5472.000,-6581.60,20061018
2,LNJ2007,LNM2007,LNN2007,LN,Livestock,122,164,185,400,12,...,8,8,8,-593.880,-10716.720,-28407.12,537.320,9696.080,25701.68,20061018
3,LNM2007,LNN2007,LNQ2007,LN,Livestock,164,185,206,400,12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20061018
4,LNZ2006,LNJ2007,LNN2007,LN,Livestock,40,122,185,400,12,...,NaN,NaN,NaN,-6040.000,NaN,NaN,8456.000,NaN,NaN,20061018
5,LNG2007,LNM2007,LNQ2007,LN,Livestock,81,164,206,400,12,...,8,8,8,-3043.560,-13637.520,-28340.52,3277.680,14686.560,30520.56,20061018
6,LCZ2006,LCG2007,LCJ2007,LC,Livestock,50,90,132,400,12,...,NaN,NaN,NaN,2328.000,NaN,NaN,-2095.200,NaN,NaN,20061018
7,LCG2007,LCJ2007,LCM2007,LC,Livestock,90,132,175,400,12,...,5,5,5,-9263.000,-18751.000,-12683.00,6669.360,13500.720,9131.76,20061018
8,LCJ2007,LCM2007,LCQ2007,LC,Livestock,132,175,219,400,12,...,52,52,52,-288.000,-3123.840,-1363.20,336.000,3644.480,1590.40,20061018
9,LCZ2006,LCJ2007,LCQ2007,LC,Livestock,50,132,219,400,12,...,NaN,NaN,NaN,-4518.640,NaN,NaN,3873.120,NaN,NaN,20061018


In [4]:
import backtesting.backtest_futures_butterfly as bfb

total_pnl_frame_1 = bfb.construct_futures_butterfly_portfolio(rule_no=1,
                                          backtest_output=backtest_output,
                                          date_list = selected_dates,
                                          pnl_field='hold_pnl5')

total_pnl_frame_2 = bfb.construct_futures_butterfly_portfolio(rule_no=2,
                                          backtest_output=backtest_output,
                                          date_list = selected_dates,
                                          pnl_field='hold_pnl5')


In [5]:
import matplotlib.pyplot as plt
#trade_data=big_data
plt.plot(total_pnl_frame_1.index,total_pnl_frame_1['portfolio'].cumsum(),
         total_pnl_frame_2.index,total_pnl_frame_2['portfolio'].cumsum())
plt.grid()
plt.legend([1,2])
plt.show()


In [8]:
import math as m
sharp_1 = m.sqrt(52)*total_pnl_frame_1['portfolio'].mean()/total_pnl_frame_1['portfolio'].std()
sharp_2 = m.sqrt(52)*total_pnl_frame_2['portfolio'].mean()/total_pnl_frame_2['portfolio'].std()

print(sharp_1)
print(sharp_2)


4.17845018843885
3.8992354929493005


In [64]:
# All tickerheads are consistently profitable when long-short rules are combined

import matplotlib.pyplot as plt
#trade_data=big_data
tickerHeadList = big_data['tickerHead'].unique()
no=19
plt.plot(total_pnl_frame_1.index,total_pnl_frame_1[tickerHeadList[no]].cumsum())
plt.grid()
plt.title(tickerHeadList[no])
plt.show()

In [ ]:
# Unproportionate Contract Weights



In [7]:
import backtesting.backtest_futures_butterfly as bfb
portfolio_contributors_output = bfb.analyze_portfolio_contributors(date_list=selected_dates,
                                   backtest_output=backtest_output,
                                  rule_no=1,
                                  date_from=20160127,
                                  date_to=20160224)

portfolio_contributors_output['ticker_head_total_pnls']

portfolio_contributors_output['total_pnl_frame']

portfolio_contributors_output['ticker_head_total_pnls']

,abs_pnl,pnl,ticker_head
CL,12563,12563,CL
HO,10190,10190,HO
NG,8833,-8833,NG
B,8711,8711,B
CC,4898,-4898,CC
FC,4805,4805,FC
RB,3795,-3795,RB
KC,3487,3487,KC
SB,2216,2216,SB
LN,1516,1516,LN


In [15]:
import math as m
m.sqrt(52)*total_pnl_frame['portfolio'].mean()/total_pnl_frame['portfolio'].std()

# Rule 1 Sharp: 4.5



4.534496285051776

In [4]:
import backtesting.backtest_stats as bs
import pandas as pd
from IPython.display import display
rank_indicators_output = bs.rank_indicators(trade_data=big_data,
                                            indicator_list = ['Q', 'QF', 'z1', 'z2', 'z3', 'z4','z5','z6','z7'],
                                            long_pnl_field='hold_pnl5long',
                                            short_pnl_field='hold_pnl5short',
                                            strategy_class='futures_butterfly',
                                            granular_ranking_type='ticker_head')
indicator_ranking_total = rank_indicators_output['indicator_ranking_total']
indicator_ranking_granular_total = rank_indicators_output['indicator_ranking_granular_total']
display(indicator_ranking_total)
display(indicator_ranking_granular_total)

# Z1,QF combo seems to provide the best results


,indicator,ranking
26,"[z1, QF]",159
66,"[z6, QF]",153
34,"[z2, QF]",153
74,"[z7, QF]",153
37,"[z2, z4]",148
70,"[z6, z4]",147
52,"[z4, z2]",145
78,"[z7, z4]",143
29,"[z1, z4]",141
58,"[z5, QF]",138


,indicator,ranking
26,"[z1, QF]",2207
34,"[z2, QF]",2132
66,"[z6, QF]",2101
13,"[Q, z4]",2096
74,"[z7, QF]",2051
29,"[z1, z4]",2044
42,"[z3, QF]",2025
58,"[z5, QF]",2014
19,"[QF, z2]",1990
24,"[QF, z7]",1974


In [3]:
import backtesting.backtest_stats as bs
import pandas as pd
from IPython.display import display


selected_data = big_data[big_data['tickerHead'] == 'B']

rank_indicators_output = bs.rank_indicators(trade_data=selected_data,
                                            indicator_list = ['Q', 'QF', 'z1', 'z2', 'z3', 'z4','z5','z6','z7'],
                                            long_pnl_field='hold_pnl5long',
                                            short_pnl_field='hold_pnl5short',
                                            strategy_class='futures_butterfly',
                                            granular_ranking_type='ticker_head')
indicator_ranking_total = rank_indicators_output['indicator_ranking_total']
indicator_ranking_granular_total = rank_indicators_output['indicator_ranking_granular_total']
display(indicator_ranking_total)
display(indicator_ranking_granular_total)


,indicator,ranking
52,"[z4, z2]",154
26,"[z1, QF]",153
34,"[z2, QF]",143
19,"[QF, z2]",142
76,"[z7, z2]",139
51,"[z4, z1]",138
66,"[z6, QF]",134
54,"[z4, z5]",133
53,"[z4, z3]",133
58,"[z5, QF]",133


,indicator,ranking
52,"[z4, z2]",154
26,"[z1, QF]",153
34,"[z2, QF]",143
19,"[QF, z2]",142
76,"[z7, z2]",139
51,"[z4, z1]",138
66,"[z6, QF]",134
54,"[z4, z5]",133
53,"[z4, z3]",133
58,"[z5, QF]",133


In [29]:
selected_data = big_data[big_data['tickerHead'] == 'CL'] 
selected_data = big_data
bs.get_indicator_rr_table(trade_data=selected_data,indicator_name='z1',
                                 strategy_class='futures_butterfly',
                                 long_pnl_field='hold_pnl5long',
                                            short_pnl_field='hold_pnl5short')

,indicator_ulimit,mean_pnl,reward_risk
0,-1.09,683.358583,0.230078
1,-0.63,279.327149,0.100574
2,-0.33,125.669351,0.046079
3,-0.08,27.877612,0.010051
4,0.14,-66.936700,-0.022964
5,0.37,152.614454,0.053610
6,0.67,429.253142,0.148988
7,1.16,532.752797,0.187186
8,NaN,796.308065,0.269843


In [37]:
selected_data = big_data[(big_data['z7'] <0)] 
selected_data = big_data 
bs.get_indicator_rr_table(trade_data=selected_data,indicator_name='z1',
                                 strategy_class='futures_butterfly',
                                 long_pnl_field='hold_pnl5long',
                                            short_pnl_field='hold_pnl5short')

,indicator_ulimit,mean_pnl,reward_risk
0,-1.09,683.358583,0.230078
1,-0.63,279.327149,0.100574
2,-0.33,125.669351,0.046079
3,-0.08,27.877612,0.010051
4,0.14,-66.936700,-0.022964
5,0.37,152.614454,0.053610
6,0.67,429.253142,0.148988
7,1.16,532.752797,0.187186
8,NaN,796.308065,0.269843


In [43]:
selected_data = big_data[(big_data['z1'] <-1.09)] 
print(selected_data['hold_pnl5long'].mean())
print(len(selected_data['hold_pnl5long']))

selected_data = big_data[(big_data['z1'] <-1.09)&(big_data['z7']<-1)] 
print(selected_data['hold_pnl5long'].mean())
print(len(selected_data['hold_pnl5long']))


689.1073932680755
19979
965.2595494610299
12150


In [21]:
selected_data = big_data[big_data['tickerHead'] == 'CL']
bs.get_indicator_rr_table(trade_data=selected_data,indicator_name='z6',
                                 strategy_class='futures_butterfly',
                                 long_pnl_field='hold_pnl5long',
                                            short_pnl_field='hold_pnl5short')

,indicator_ulimit,mean_pnl,reward_risk
0,-1.00,1382.932433,0.385648
1,-0.57,453.584165,0.126608
2,-0.27,-23.147689,-0.006337
3,-0.04,-60.149856,-0.017051
4,0.16,-256.562493,-0.071857
5,0.37,385.870620,0.125330
6,0.62,417.644916,0.127393
7,1.01,808.313675,0.259075
8,NaN,1403.266237,0.437179


In [22]:
selected_data = big_data[big_data['tickerHead'] == 'CL']
bs.get_indicator_rr_table(trade_data=selected_data,indicator_name='z7',
                                 strategy_class='futures_butterfly',
                                 long_pnl_field='hold_pnl5long',
                                            short_pnl_field='hold_pnl5short')

,indicator_ulimit,mean_pnl,reward_risk
0,-0.99,1356.025236,0.379541
1,-0.55,396.119573,0.109960
2,-0.24,-31.871966,-0.008778
3,-0.02,9.509402,0.002751
4,0.19,-317.944748,-0.089190
5,0.39,312.093239,0.102892
6,0.65,432.325998,0.132453
7,1.03,711.264618,0.223584
8,NaN,1477.046228,0.464357


In [8]:
import backtesting.backtest_stats as bs
selected_data = big_data[(big_data['tickerHead'] == 'CL')&(big_data['z7'] <0)]
selected_data = big_data
bs.get_indicator_rr_double_table(trade_data=selected_data,indicator_list=['z1','QF'],
                                 strategy_class='futures_butterfly',
                                 long_pnl_field='long_pnl_ws3',
                                            short_pnl_field='short_pnl_ws3')

,indicator1_ulimit,indicator2_ulimit,mean_pnl,reward_risk
0,-0.21,10,-154277.722703,-2.851870
1,-0.21,42,-156105.503443,-2.798739
2,-0.21,NaN,-111740.683120,-1.938704
3,0.56,30,-121380.738139,-2.794428
4,0.56,70,-135005.000496,-3.048926
5,0.56,NaN,-97979.353126,-2.109227
6,NaN,54,-40112.355805,-0.946086
7,NaN,87,-40049.070473,-0.914735
8,NaN,NaN,-41666.042185,-0.888070


In [6]:
selected_data = big_data
bs.get_indicator_rr_double_table(trade_data=selected_data,indicator_list=['z1','QF'],
                                 strategy_class='futures_butterfly',
                                 long_pnl_field='long_pnl20',
                                            short_pnl_field='short_pnl20')

,indicator1_ulimit,indicator2_ulimit,mean_pnl,reward_risk
0,-0.34,13,1319.331228,0.300907
1,-0.34,48,642.908351,0.135953
2,-0.34,NaN,-105.100797,-0.019360
3,0.37,28,331.306475,0.071836
4,0.37,67,-301.701946,-0.059021
5,0.37,NaN,-649.913315,-0.120762
6,NaN,50,584.972357,0.106401
7,NaN,85,1137.348948,0.253841
8,NaN,NaN,2425.662114,0.662668


In [6]:
import backtesting.backtest_stats as bs
selected_data = big_data[big_data['tickerHead'] == 'CL']
selected_data = big_data
bs.get_indicator_rr_double_table(trade_data=selected_data,indicator_list=['z1','QF'],
                                 strategy_class='futures_butterfly',
                                 long_pnl_field='long_pnl20',
                                            short_pnl_field='short_pnl20')

,indicator1_ulimit,indicator2_ulimit,mean_pnl,reward_risk
0,-0.34,13,1319.331228,0.300907
1,-0.34,48,642.908351,0.135953
2,-0.34,NaN,-105.100797,-0.019360
3,0.37,28,331.306475,0.071836
4,0.37,67,-301.701946,-0.059021
5,0.37,NaN,-649.913315,-0.120762
6,NaN,50,584.972357,0.106401
7,NaN,85,1137.348948,0.253841
8,NaN,NaN,2425.662114,0.662668


In [6]:
selected_data = big_data
long_data = selected_data[(selected_data['z1']<-1.2)&(selected_data['QF']<13)]

print(long_data['long_pnl20'].mean()/long_data['long_pnl20'].std())
print(long_data['long_pnl40'].mean()/long_data['long_pnl40'].std())
print(long_data['long_pnl60'].mean()/long_data['long_pnl60'].std())
print(long_data['long_pnl_ws1'].mean()/long_data['long_pnl_ws1'].std())
print(long_data['long_pnl_ws2'].mean()/long_data['long_pnl_ws2'].std())
print(long_data['long_pnl_ws3'].mean()/long_data['long_pnl_ws3'].std())

0.23048797003322014
0.2932597839513357
0.2507936234511511
0.37117197485724723
0.5359296990032697
0.6931755779396542


In [7]:
selected_data = big_data
short_data = selected_data[(selected_data['z1']>0.6)&(selected_data['QF']>85)]

print(short_data['short_pnl20'].mean()/short_data['short_pnl20'].std())
print(short_data['short_pnl40'].mean()/short_data['short_pnl40'].std())
print(short_data['short_pnl60'].mean()/short_data['short_pnl60'].std())
print(short_data['short_pnl_ws1'].mean()/short_data['short_pnl_ws1'].std())
print(short_data['short_pnl_ws2'].mean()/short_data['short_pnl_ws2'].std())
print(short_data['short_pnl_ws3'].mean()/short_data['short_pnl_ws3'].std())

0.2456404898628928
0.28700719110643425
0.3294311383350736
-1.2763544070534736
-1.2762887061626706
-1.2762887061626706


In [5]:
import matplotlib.pyplot as plt
#trade_data=big_data
plt.scatter(long_data['long_pnl20'],long_data['long_pnl_ws1'])
plt.grid()
plt.show()

In [11]:
list(range(1,7))

[1, 2, 3, 4, 5, 6]

In [6]:
long_data['long_pnl_ws1'].min()

-70618.49999999999

In [26]:
wuhu=long_data[long_data['long_pnl_ws1']<long_data['long_pnl20']-10000]
wuhu['trDte1']
wuhu['long_stop_days3']
wuhu['bf_price']
wuhu['short_pnl_ws3']
wuhu.columns
wuhu['second_spread_weight_1']

#wuhu['long_pnl_ws3']
#wuhu['report_date']
# continue with 5
selected_trade = wuhu.iloc[2]
selected_trade


ticker1                      LNN2007
ticker2                      LNQ2007
ticker3                      LNV2007
tickerHead                        LN
tickerClass                Livestock
trDte1                           133
trDte2                           154
trDte3                           196
multiplier                       400
agg                               12
cBack                             20
Q                                 13
QF                                10
z1                             -1.21
z2                             -1.88
z3                             -1.23
z4                             -0.48
z5                             -1.23
z6                             -1.21
z7                             -1.23
r1                             37.96
r2                              1.89
RC                              0.01
seasonality                        0
yield1                         1.366
yield2                         9.614
bf_price                       -5.15
b

In [27]:
import opportunity_constructs.utilities as opu
aligned_data_output = opu.get_aligned_futures_data(contract_list=['LNN2007','LNQ2007','LNV2007'],
                                                   aggregation_method=12,
                                                   contracts_back=2,
                                                   date_to=20070403)

aligned_data = aligned_data_output['aligned_data']
bf_price = aligned_data['c1']['close_price']-2*aligned_data['c2']['close_price']+aligned_data['c3']['close_price']
bf_price


import matplotlib.pyplot as plt
#trade_data=big_data
plt.plot(bf_price)
plt.grid()
plt.show()
aligned_data['c1']




,ticker,ticker_head,ticker_month,ticker_year,cal_dte,tr_dte,open_price,high_price,low_price,close_price,volume,cont_indx,change1,change1_instant,change2,change5,change10,change20,change_5,change_1
settle_date,,,,,,,,,,,,,,,,,,,,
2006-01-23,LNN2006,LN,7,2006,175,121,67.95,67.95,66.80,67.450,867,200607,-0.420,-0.980,-0.070,0.730,0.250,1.680,-0.200,-0.450
2006-01-24,LNN2006,LN,7,2006,174,120,67.40,67.40,66.30,66.470,524,200607,0.350,-0.420,0.650,0.800,1.200,2.420,-1.600,-0.980
2006-01-25,LNN2006,LN,7,2006,173,119,66.40,66.40,65.40,66.050,636,200607,0.300,0.350,0.670,-0.750,1.000,1.770,-0.800,-0.420
2006-01-26,LNN2006,LN,7,2006,172,118,65.75,66.45,65.70,66.400,368,200607,0.370,0.300,0.500,-0.330,1.270,1.800,-1.170,0.350
2006-01-27,LNN2006,LN,7,2006,171,117,66.40,66.80,66.15,66.700,1042,200607,0.130,0.370,-0.220,-0.400,0.780,1.250,-1.200,0.300
2006-01-30,LNN2006,LN,7,2006,168,116,66.45,67.07,66.45,67.070,714,200607,-0.350,0.130,-1.550,-0.480,0.920,2.020,-0.380,0.370
2006-01-31,LNN2006,LN,7,2006,167,115,67.10,67.35,66.80,67.200,463,200607,-1.200,-0.350,-0.480,0.400,0.320,2.070,0.730,0.130
2006-02-01,LNN2006,LN,7,2006,166,114,67.10,67.45,66.80,66.850,502,200607,0.720,-1.200,1.020,1.750,2.250,3.570,0.800,-0.350
2006-02-02,LNN2006,LN,7,2006,165,113,66.25,66.45,65.05,65.650,640,200607,0.300,0.720,0.350,1.600,2.100,2.250,-0.750,-1.200


In [95]:
logical_index =  bf_price>0.5
logical_index

settle_date
2006-04-05    False
2006-04-06    False
2006-04-07    False
2006-04-10    False
2006-04-11    False
2006-04-12    False
2006-04-13    False
2006-04-17    False
2006-04-18    False
2006-04-19    False
2006-04-20    False
2006-04-21    False
2006-04-24    False
2006-04-25    False
2006-04-26    False
2006-04-27    False
2006-04-28    False
2006-05-01    False
2006-05-02    False
2006-05-03    False
2006-05-04    False
2006-05-05    False
2006-05-08    False
2006-05-09    False
2006-05-10    False
2006-05-11    False
2006-05-12    False
2006-05-15    False
2006-05-16    False
2006-05-17    False
              ...  
2007-05-04    False
2007-05-07    False
2007-05-08    False
2007-05-09     True
2007-05-10    False
2007-05-11    False
2007-05-14    False
2007-05-15    False
2007-05-16    False
2007-05-17    False
2007-05-18    False
2007-05-21    False
2007-05-22     True
2007-05-23    False
2007-05-24    False
2007-05-25    False
2007-05-29    False
2007-05-30    False
2007-05-

In [96]:

logical_index2 = logical_index.copy()

for i in range(len(logical_index)):
    
    if logical_index.iloc[i]:
        logical_index2.iloc[i-1]= True
        logical_index2.iloc[i+1]= True
        
        

In [41]:
logical_index2

settle_date
2003-02-25    False
2003-02-26    False
2003-02-27    False
2003-02-28    False
2003-03-03    False
2003-03-04    False
2003-03-05    False
2003-03-06    False
2003-03-07    False
2003-03-10    False
2003-03-11    False
2003-03-12    False
2003-03-13    False
2003-03-14    False
2003-03-17    False
2003-03-18    False
2003-03-19    False
2003-03-20    False
2003-03-21    False
2003-03-24    False
2003-03-25    False
2003-03-26    False
2003-03-27    False
2003-03-28    False
2003-03-31    False
2003-04-01    False
2003-04-02    False
2003-04-03    False
2003-04-04    False
2003-04-07    False
              ...  
2007-03-09    False
2007-03-13    False
2007-03-16    False
2007-03-19    False
2007-03-21    False
2007-03-22    False
2007-03-23    False
2007-03-26    False
2007-03-28    False
2007-03-30    False
2007-04-02    False
2007-04-05    False
2007-04-09    False
2007-04-10    False
2007-04-11    False
2007-04-13    False
2007-04-16    False
2007-04-17    False
2007-04-

In [102]:
kuzu = aligned_data.loc[logical_index2,'c3'][['close_price','ticker']]
kuzu.iloc[0:3]

,close_price,ticker
settle_date,,
2007-04-10,68.25,CLZ2011
2007-04-11,70.00,CLZ2011
2007-04-12,68.52,CLZ2011


In [10]:
import get_price.get_futures_price as gfp
report_date = 20110126
ticker_list = ['CCK2011','CCN2011','CCU2011']

risk_per_trade = 10000
quantity_long = round(risk_per_trade/abs(selected_trade['downside']))
quantity_short = -round(risk_per_trade/abs(selected_trade['upside']))
data_list = []


for j in range(3):
    ticker_frame = gfp.get_futures_price_preloaded(ticker=ticker_list[j],
                                                   settle_date_from_exclusive=report_date)
    ticker_frame.set_index('settle_date', drop=False, inplace=True)
    data_list.append(ticker_frame)
merged_data = pd.concat(data_list, axis=1, join='inner')

spread_price = (merged_data['close_price'].iloc[:,0]-merged_data['close_price'].iloc[:,1])-selected_trade['second_spread_weight_1']*(merged_data['close_price'].iloc[:,1]-merged_data['close_price'].iloc[:,2])

long_path = quantity_long*selected_trade['multiplier']*(spread_price[1:]-spread_price[0])
short_path = quantity_short*selected_trade['multiplier']*(spread_price[1:]-spread_price[0])


In [80]:
merged_data[merged_data['tr_dte'].iloc[:,0]>=15]

,settle_date,ticker,ticker_head,ticker_month,ticker_year,cal_dte,tr_dte,open_price,high_price,low_price,...,volume,cont_indx,change1,change1_instant,change2,change5,change10,change20,change_5,change_1
settle_date,,,,,,,,,,,,,,,,,,,,,
2011-01-27,2011-01-27,CCK2011,CC,5,2011,106,74,3322,3390,3322,...,261,201109,83,-61,51,22,110,325,106,-13
2011-01-28,2011-01-28,CCK2011,CC,5,2011,105,73,3335,3342,3250,...,340,201109,-32,83,15,-86,39,196,61,-61
2011-01-31,2011-01-31,CCK2011,CC,5,2011,102,72,3260,3367,3226,...,444,201109,47,-32,58,-36,40,265,72,83
2011-02-01,2011-02-01,CCK2011,CC,5,2011,101,71,3366,3371,3283,...,410,201109,11,47,-76,-60,23,281,29,-32
2011-02-02,2011-02-02,CCK2011,CC,5,2011,100,70,3345,3376,3330,...,740,201109,-87,11,-112,3,25,197,24,47
2011-02-03,2011-02-03,CCK2011,CC,5,2011,99,69,3374,3385,3349,...,719,201109,-25,-87,-7,88,143,308,48,11
2011-02-04,2011-02-04,CCK2011,CC,5,2011,98,68,3369,3369,3250,...,683,201109,18,-25,41,125,220,304,22,-87
2011-02-07,2011-02-07,CCK2011,CC,5,2011,95,67,3280,3288,3199,...,500,201109,23,18,97,76,247,227,-86,-25
2011-02-08,2011-02-08,CCK2011,CC,5,2011,94,66,3238,3275,3220,...,574,201109,74,23,72,83,214,125,-36,18


In [76]:
import matplotlib.pyplot as plt
#trade_data=big_data
plt.plot(merged_data['close_price'].iloc[:,2])
plt.grid()
plt.show()

In [4]:
# For CL filtering based on z6 makes sense. [-0.6; 0.3]



selected_data = big_data[big_data['tickerHead'] == 'CL']
#selected_data = big_data


selected_data2 = selected_data[(selected_data['z1'] <-1.2)&(selected_data['QF'] <=12)] 
print(selected_data2['hold_pnl5long'].mean())
print(len(selected_data2['hold_pnl5long']))

selected_data2 = selected_data[(selected_data['z1'] <-1.2)&(selected_data['QF'] <=12)&(selected_data['z6']<0)
                               &(selected_data['z6']>-0.6)]

print(selected_data2['hold_pnl10long'].mean())
print(len(selected_data2['hold_pnl10long']))

selected_data2 = selected_data[(selected_data['z1'] >0.6)&(selected_data['QF'] >=85)] 
print(selected_data2['hold_pnl10short'].mean())
print(len(selected_data2['hold_pnl10short']))

selected_data2 = selected_data[(selected_data['z1'] >0.6)&(selected_data['QF'] >=85)&(selected_data['z6']<0.3)] 
print(selected_data2['hold_pnl10short'].mean())
print(len(selected_data2['hold_pnl10short']))

1727.9728866624232
1634
1533.3950000002924
101
1837.6201463140467
3587
987.0799999999966
160


In [62]:
# LONG TRADES

# CL,B z6<-0.6

#selected_data = big_data[big_data['tickerHead'] == 'B']
selected_data
selected_data1 = selected_data[(selected_data['z1'] <-1.2)&(selected_data['QF'] <=12)] 
print('per contract pnl: ' + str(selected_data1['hold_pnl10_per_contract'].mean()))
print('sharp: ' + str(selected_data1['hold_pnl10_per_contract'].mean()/selected_data1['hold_pnl10_per_contract'].std()))
print('num trades: ' + str(len(selected_data1['hold_pnl10_per_contract'])))

print(30*'-')
selected_data2 = selected_data[(selected_data['z1'] <-1.2)&(selected_data['QF'] <=12)&(selected_data['z5'] < -1.2)] 
print('per contract pnl: ' + str(selected_data2['hold_pnl10_per_contract'].mean()))
print('sharp: ' + str(selected_data2['hold_pnl10_per_contract'].mean()/selected_data2['hold_pnl10_per_contract'].std()))
print('num trades: ' + str(len(selected_data2['hold_pnl10_per_contract'])))

print(30*'-')
selected_data3 = selected_data[(selected_data['z1'] <-1.2)&(selected_data['QF'] <=12)&(selected_data['z5'] > -0.6)] 
print('per contract pnl: ' + str(selected_data3['hold_pnl10_per_contract'].mean()))
print('sharp: ' + str(selected_data3['hold_pnl10_per_contract'].mean()/selected_data3['hold_pnl10_per_contract'].std()))
print('num trades: ' + str(len(selected_data3['hold_pnl10_per_contract'])))




per contract pnl: 3.486759545411965
sharp: 0.14625693851240124
num trades: 757
------------------------------
per contract pnl: 3.372427804002262
sharp: 0.1400865464468809
num trades: 734
------------------------------
per contract pnl: nan
sharp: nan
num trades: 0


In [72]:
selected_data = big_data[big_data['tickerHead'] == 'B']
selected_data = selected_data[selected_data['second_spread_weight_1']>2]
selected_data1 = selected_data[(selected_data['z1'] <-1.2)&(selected_data['QF'] <=12)] 
print('per contract pnl: ' + str(selected_data1['hold_pnl10_per_contract'].mean()))
print('sharp: ' + str(selected_data1['hold_pnl10_per_contract'].mean()/selected_data1['hold_pnl10_per_contract'].std()))
print('num trades: ' + str(len(selected_data1['hold_pnl10_per_contract'])))

per contract pnl: 0.24846375803704318
sharp: 0.031198208127495367
num trades: 17


In [56]:
# SHORT TRADES

# CL,B z6>0.3

selected_data = big_data[big_data['tickerHead'] == 'B']
selected_data1 = selected_data[(selected_data['z1'] >0.6)&(selected_data['QF'] >=85)] 
print('per contract pnl: ' + str(selected_data1['hold_pnl10_per_contract'].mean()))
print('sharp: ' + str(selected_data1['hold_pnl10_per_contract'].mean()/selected_data1['hold_pnl10_per_contract'].std()))
print('num trades: ' + str(len(selected_data1['hold_pnl10_per_contract'])))

print(30*'-')
selected_data2 = selected_data[(selected_data['z1']>1.2)&(selected_data['QF'] >=85)&(selected_data['z5']<0.6)] 
print('per contract pnl: ' + str(selected_data2['hold_pnl10_per_contract'].mean()))
print('sharp: ' + str(selected_data2['hold_pnl10_per_contract'].mean()/selected_data2['hold_pnl10_per_contract'].std()))
print('num trades: ' + str(len(selected_data2['hold_pnl10_per_contract'])))




per contract pnl: 5.433852598540892
sharp: 0.25593859605386765
num trades: 2199
------------------------------
per contract pnl: nan
sharp: nan
num trades: 0


In [54]:
selected_data = big_data[big_data['tickerHead'] == 'B']
selected_data = selected_data[(selected_data['trDte2']-selected_data['trDte1']>100)&
                              (selected_data['trDte3']-selected_data['trDte2']>100)]
selected_data1 = selected_data[(selected_data['z1'] >0.6)&(selected_data['QF'] >=85)] 
print('per contract pnl: ' + str(selected_data1['hold_pnl10_per_contract'].mean()))
print('sharp: ' + str(selected_data1['hold_pnl10_per_contract'].mean()/selected_data1['hold_pnl10_per_contract'].std()))
print('num trades: ' + str(len(selected_data1['hold_pnl10_per_contract'])))

print(30*'-')
selected_data2 = selected_data[(selected_data['z1']>1.2)&(selected_data['QF'] >=85)&(selected_data['z5']<0.6)] 
print('per contract pnl: ' + str(selected_data2['hold_pnl10_per_contract'].mean()))
print('sharp: ' + str(selected_data2['hold_pnl10_per_contract'].mean()/selected_data2['hold_pnl10_per_contract'].std()))
print('num trades: ' + str(len(selected_data2['hold_pnl10_per_contract'])))

per contract pnl: 10.981330542200567
sharp: 0.2384929550187611
num trades: 261
------------------------------
per contract pnl: nan
sharp: nan
num trades: 0


In [57]:
import matplotlib.pyplot as plt
#trade_data=big_data
plt.plot(selected_data1['hold_pnl10_per_contract'].cumsum())
plt.grid()
plt.show()

In [32]:
selected_data2

,ticker1,ticker2,ticker3,tickerHead,tickerClass,trDte1,trDte2,trDte3,multiplier,agg,...,hold_pnl2,hold_pnl5,hold_pnl10,hold_pnl20,hold_pnl1_per_contract,hold_pnl2_per_contract,hold_pnl5_per_contract,hold_pnl10_per_contract,hold_pnl20_per_contract,mom5


In [98]:
selected_data = big_data[big_data['tickerHead'] == 'CL']

selected_data2 = selected_data[(selected_data['z1'] <-1.2)] 
print('Avg Long Pnl:' + str(selected_data2['hold_pnl5long'].mean()))
print(len(selected_data2['hold_pnl5long']))

selected_data2 = selected_data[(selected_data['z1'] <-1.2)&(selected_data['z6']>-0.6)] 
print('Avg Long Pnl Filtered:' + str(selected_data2['hold_pnl5long'].mean()))
print(len(selected_data2['hold_pnl5long']))

selected_data2 = selected_data[(selected_data['z1'] >0.6)] 
print('Avg Short Pnl:' + str(selected_data2['hold_pnl5short'].mean()))
print(len(selected_data2['hold_pnl5short']))

selected_data2 = selected_data[(selected_data['z1'] >0.6)&(selected_data['z6']<0)] 
print('Avg Short Pnl Filtered:' +str(selected_data2['hold_pnl5short'].mean()))
print(len(selected_data2['hold_pnl5short']))

Avg Long Pnl:1286.8930856219624
3190
Avg Long Pnl Filtered:222.96000000014857
256
Avg Short Pnl:982.9561072988266
8073
Avg Short Pnl Filtered:414.9526315789886
76


In [37]:

# Can we use momentum?
# As a Broad Measure This is not correct

import backtesting.backtest_stats as bs

import pandas as pd
from IPython.display import display

tickerHeadList = big_data['tickerHead'].unique()

no=1
trade_data = big_data
trade_data = big_data[big_data['mean_reversion_signif']==1]
#trade_data = big_data[big_data['mom5']>0]



#trade_data = trade_data[trade_data['tickerHead']==tickerHeadList[no]]
#trade_data = trade_data[trade_data['tickerHead']=='NG']
#print(tickerHeadList[no])

#S long side and RB long side were not all that profitable

pnl_field = 'hold_pnl10'


standard_short_stats1 = bs.get_summary_stats(trade_data[(trade_data['z1']>=0.6) & 
                                                     (trade_data['z1']<1.2) &
                                                      (trade_data['QF']>=85)][pnl_field].values)
standard_short_stats2 = bs.get_summary_stats(trade_data[(trade_data['z1']>=1.2) &
                                                      (trade_data['QF']>=85)][pnl_field].values)
    

standard_long_stats2 = bs.get_summary_stats(trade_data[(trade_data['z1']<=-1.2) &
                                                      (trade_data['QF']<=12)][pnl_field].values)
    
summary_frame = pd.DataFrame.from_items([('tier', ['short1','short2','long2']), 
                         ('reward_risk', [standard_short_stats1['reward_risk'],
                                         standard_short_stats2['reward_risk'],
                                         standard_long_stats2['reward_risk']]),                
                         ('mean_pnl', [standard_short_stats1['mean_pnl'],
                                         standard_short_stats2['mean_pnl'],
                                         standard_long_stats2['mean_pnl']]),                     
                        ('total_pnl', [standard_short_stats1['total_pnl'],
                                         standard_short_stats2['total_pnl'],
                                         standard_long_stats2['total_pnl']])])
display(summary_frame)

# waiting for a month seems to provide better results although this is more true for short then longs

,tier,reward_risk,mean_pnl,total_pnl
0,short1,0.595983,1624.980363,10815869.2940
1,short2,0.674250,2124.695807,13706412.6535
2,long2,0.506919,1712.888050,10265338.0810


In [5]:
import backtesting.backtest_stats as bs

import pandas as pd
from IPython.display import display

no=20
trade_data = big_data
trade_data = big_data[big_data['tickerHead']==tickerHeadList[no]]
print(tickerHeadList[no])

#S long side and RB long side were not all that profitable

pnl_field_list = ['hold_pnl5','hold_pnl10','hold_pnl20']

for pnl_field in pnl_field_list:
    standard_short_stats1 = bs.get_summary_stats(trade_data[(trade_data['z1']>=0.6) & 
                                                     (trade_data['z1']<1.2) &
                                                      (trade_data['QF']>=85)][pnl_field].values)
    standard_short_stats2 = bs.get_summary_stats(trade_data[(trade_data['z1']>=1.2) &
                                                      (trade_data['QF']>=85)][pnl_field].values)
    
    standard_long_stats1 = bs.get_summary_stats(trade_data[(trade_data['z1']<=-0.6) & 
                                                     (trade_data['z1']>-1.2) &
                                                      (trade_data['QF']<=12)][pnl_field].values)
    
    standard_long_stats2 = bs.get_summary_stats(trade_data[(trade_data['z1']<=-1.2) &
                                                      (trade_data['QF']<=12)][pnl_field].values)
    
    summary_frame = pd.DataFrame.from_items([('tier', ['short1','short2','long1','long2']), 
                         ('reward_risk', [standard_short_stats1['reward_risk'],
                                         standard_short_stats2['reward_risk'],
                                         standard_long_stats1['reward_risk'],
                                         standard_long_stats2['reward_risk']]),                
                         ('mean_pnl', [standard_short_stats1['mean_pnl'],
                                         standard_short_stats2['mean_pnl'],
                                         standard_long_stats1['mean_pnl'],
                                         standard_long_stats2['mean_pnl']]),                     
                        ('total_pnl', [standard_short_stats1['total_pnl'],
                                         standard_short_stats2['total_pnl'],
                                         standard_long_stats1['total_pnl'],
                                         standard_long_stats2['total_pnl']])])
    display(summary_frame)

# waiting for a month seems to provide better results although this is more true for short then longs

IndexError: index 20 is out of bounds for axis 0 with size 20

In [85]:
big_data['second_spread_weight_1'].quantile([0.1, 0.25, 0.5, 0.75, 0.99])


array(['B', 'BO', 'C', 'CC', 'CL', 'CT', 'ED', 'FC', 'HO', 'KC', 'KW',
       'LC', 'LN', 'NG', 'OJ', 'RB', 'S', 'SB', 'SM', 'W'], dtype=object)

In [7]:
import backtesting.backtest_stats as bs
import pandas as pd
from IPython.display import display

#trade_data = big_data[big_data['tickerClass']=='Energy']
# can trade all the way to three times of one contract versus the other, the results does't seem
# to be effected by that all that much!

trade_data = big_data
trade_data = trade_data[trade_data['second_spread_weight_1']>2]

pnl_field_list = ['hold_pnl5_per_contract','hold_pnl10_per_contract','hold_pnl20_per_contract']

for pnl_field in pnl_field_list:
    standard_short_stats1 = bs.get_summary_stats(trade_data[(trade_data['z1']>=0.6) & 
                                                     (trade_data['z1']<1.2) &
                                                      (trade_data['QF']>=85)][pnl_field].values)
    standard_short_stats2 = bs.get_summary_stats(trade_data[(trade_data['z1']>=1.2) &
                                                      (trade_data['QF']>=85)][pnl_field].values)
    
    standard_long_stats1 = bs.get_summary_stats(trade_data[(trade_data['z1']<=-0.6) & 
                                                     (trade_data['z1']>-1.2) &
                                                      (trade_data['QF']<=12)][pnl_field].values)
    
    standard_long_stats2 = bs.get_summary_stats(trade_data[(trade_data['z1']<=-1.2) &
                                                      (trade_data['QF']<=12)][pnl_field].values)
    
    summary_frame = pd.DataFrame.from_items([('tier', ['short1','short2','long1','long2']), 
                         ('reward_risk', [standard_short_stats1['reward_risk'],
                                         standard_short_stats2['reward_risk'],
                                         standard_long_stats1['reward_risk'],
                                         standard_long_stats2['reward_risk']]),                
                         ('mean_pnl', [standard_short_stats1['mean_pnl'],
                                         standard_short_stats2['mean_pnl'],
                                         standard_long_stats1['mean_pnl'],
                                         standard_long_stats2['mean_pnl']]),                     
                        ('total_pnl', [standard_short_stats1['total_pnl'],
                                         standard_short_stats2['total_pnl'],
                                         standard_long_stats1['total_pnl'],
                                         standard_long_stats2['total_pnl']])])
    display(summary_frame)

# waiting for a month seems to provide better results although this is more true for short then longs


,tier,reward_risk,mean_pnl,total_pnl
0,short1,0.119796,1.270421,321.416565
1,short2,0.746806,11.887401,4136.815640
2,long1,-0.001211,-0.010544,-5.324736
3,long2,0.283114,3.392508,1265.405657


,tier,reward_risk,mean_pnl,total_pnl
0,short1,0.204282,2.575923,654.284497
1,short2,0.163211,5.127525,1789.506134
2,long1,-0.200893,-2.545899,-1290.770885
3,long2,0.522997,6.779586,2549.124419


,tier,reward_risk,mean_pnl,total_pnl
0,short1,0.729487,8.716445,2213.977018
1,short2,0.422679,12.239052,4295.907208
2,long1,-0.116668,-2.032356,-1030.404355
3,long2,0.896204,13.701313,5151.693737


In [22]:
# some interesting results when you compare these iwth fixed holding periods
import backtesting.backtest_stats as bs
import pandas as pd
from IPython.display import display

trade_data = big_data


pnl_field_list = ['path_pnl5_per_contract',
                  'path_pnl10_per_contract',
                  'path_pnl15_per_contract',
                  'path_pnl20_per_contract',
                  'path_pnl25_per_contract']

for pnl_field in pnl_field_list:
    standard_short_stats1 = bs.get_summary_stats(trade_data[(trade_data['z1']>=0.6) & 
                                                     (trade_data['z1']<1.2) &
                                                      (trade_data['QF']>=85)][pnl_field].values)
    standard_short_stats2 = bs.get_summary_stats(trade_data[(trade_data['z1']>=1.2) &
                                                      (trade_data['QF']>=85)][pnl_field].values)
    
    standard_long_stats1 = bs.get_summary_stats(trade_data[(trade_data['z1']<=-0.6) & 
                                                     (trade_data['z1']>-1.2) &
                                                      (trade_data['QF']<=12)][pnl_field].values)
    
    standard_long_stats2 = bs.get_summary_stats(trade_data[(trade_data['z1']<=-1.2) &
                                                      (trade_data['QF']<=12)][pnl_field].values)
    
    summary_frame = pd.DataFrame.from_items([('tier', ['short1','short2','long1','long2']), 
                         ('reward_risk', [standard_short_stats1['reward_risk'],
                                         standard_short_stats2['reward_risk'],
                                         standard_long_stats1['reward_risk'],
                                         standard_long_stats2['reward_risk']]),
                          ('mean_pnl', [standard_short_stats1['mean_pnl'],
                                         standard_short_stats2['mean_pnl'],
                                         standard_long_stats1['mean_pnl'],
                                         standard_long_stats2['mean_pnl']]),                                   
                        ('total_pnl', [standard_short_stats1['total_pnl'],
                                         standard_short_stats2['total_pnl'],
                                         standard_long_stats1['total_pnl'],
                                         standard_long_stats2['total_pnl']])])
    display(summary_frame)

,tier,reward_risk,mean_pnl,total_pnl
0,short1,3.460401,5.174432,36329.683649
1,short2,3.795920,9.489801,66893.606245
2,long1,0.259133,1.231704,9114.611967
3,long2,2.554748,9.449739,63747.939878


,tier,reward_risk,mean_pnl,total_pnl
0,short1,3.396150,6.080134,42664.300782
1,short2,3.215697,10.900496,76826.692738
2,long1,0.162136,1.134242,8393.390407
3,long2,1.813641,9.915478,66869.980800


,tier,reward_risk,mean_pnl,total_pnl
0,short1,2.546106,6.686132,46916.586174
1,short2,2.359706,11.699662,82447.519486
2,long1,0.142708,1.281695,9484.540121
3,long2,1.466741,10.345209,69747.400584


,tier,reward_risk,mean_pnl,total_pnl
0,short1,2.013489,7.031633,49305.810505
1,short2,2.159577,12.380557,87208.643814
2,long1,0.107109,1.093764,8091.668421
3,long2,1.189466,10.313393,69532.898312


,tier,reward_risk,mean_pnl,total_pnl
0,short1,1.710191,7.040760,49355.726502
1,short2,2.043907,12.774421,89957.470671
2,long1,0.089112,1.012456,7490.145944
3,long2,1.088333,10.527027,70962.688875


In [6]:
long_trades['second_spread_weight_1']

186    0.53
206    0.72
207    0.73
261    0.96
274    1.09
287    0.33
299    1.11
300    0.84
305    0.55
310    0.31
312    1.36
313    0.90
318    0.60
323    0.26
329    0.29
330    0.25
Name: second_spread_weight_1, dtype: float64